In [1]:
import pickle
import os

from glob import glob
from gensim import models
from nltk.corpus import stopwords
from FlaskApp.transform import *
from googletrans import Translator

import pandas as pd
import numpy as np

In [62]:
Translator().detect('tentang pekerjaan ini more than 50 000 people 49 countries over 120 office locations we re not just engineers we re global team of data scientists consultants construction workers and innovators all working to create better tomorrow every day we help customers in energy chemicals and resources get one step closer to solving our planet toughest issues climate change the energy transition digital transformation and how we can deliver more sustainable world whatever your ambition there path for you here and there no barrier to your potential career success for more information visit worley com key accountabilities responsible for safe working practices both personally and in their designs using the worley framework ensure they understand and apply the worley quality management system ensure design work conforms to applicable specifications codes and standards ensure checking procedures are adhered to before issue of drawings and documents to ensure no rework due to errors in design prepare engineering drawings and documentation promote interdisciplinary communications on projects ensure project scope budgets and schedules for design and drafting work are effectively communicated and met on projects control the allocation of drafting work to junior designers in the project mentoring and supervision of junior designers requirements degree diploma of engineering in the relevant field or equivalent minimum 5 years experience in 3d civil design for earthworks knowledge in design for constructability commissioning operations and maintenance have the experience and skills to set the standard and drive the rest of the team to excel in hse competent in civil 3d cad software excellent communication skills work as part of multi discipline team excellent time management skills and the ability to work autonomously must be able to relocate to project site to work what we offer you providing an agile culture and challenging work opportunities results in positive work environment our people enjoy competitive compensation packages and fun personal collaborative and safe working environment we value equal employment opportunity and are committed to promoting fairness equality and diversity our policy is to conduct background checks for all candidates who accept an offer of employment with us if you have the passion and talent to keep up it time to test the limits of what you can become find your future with worley company worley primary location idn sulawesi job civil schedule full time employment type employee job level executive job posting apr 17 2023 unposting date may 17 2023 reporting manager title study manager').lang

'en'

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
len(os.listdir('data/scraped'))

15

In [5]:
filepaths = glob('data/scraped/*.pickle')

In [6]:
jobs = []
for filepath in filepaths:
    filename = filepath.split('\\')[-1]
    with open(filepath, 'rb') as file:
        loaded = pickle.load(file)
        for job in loaded:
            if 'Type' not in job.keys():
                print(job['Title'])
            jobs.append(job)

Arsitek Jawa Tengah


In [91]:
len(loaded)

5125

In [92]:
len([i for i in loaded if i['Language'] == 'id'])

783

In [74]:
# add language
translator = Translator()

for job in jobs:
    if 'Language' not in job.keys():
        lang = translator.detect(job['Description']).lang
        job['Language'] = lang
        print(job['Title'], 'language added!')

Receptionist language added!
Server language added!
Bartender language added!
Receptionist GA language added!
Server language added!
Bartender language added!
Supervisor-Restaurant language added!
Chef-Specialty Restaurant language added!
Commis Chef II, Japanese language added!
Mgr-Food & Beverage II-D language added!
Waiter/ss language added!
Supervisor-Stewards language added!
Supervisor-Beverage language added!
Supervisor-Food & Beverage language added!
Paper Machine Supervisor PM1 language added!
Pick Up Supervisor Surabaya language added!
Server language added!
Executive Assistant language added!
Expert Front End Engineer (Marketplace Order) - Sea Labs language added!
Supervisor Consumer Engagement - Semarang language added!
Supervisor Produksi language added!
AsstMgr-Front Office language added!
Waiter – Daily Worker language added!
Sales Engineer language added!
Bartendress language added!
Server language added!
Daily Worker Waiter/Waitress language added!
Waiter / Waitress lan

In [77]:
n = 0
for job in jobs:
    if 'Language' in job.keys():
        n += 1

In [78]:
n, len(jobs)

(5125, 5125)

In [81]:
# filename = 'all_jobs'

# with open(f'data/scraped/{filename}.pickle', 'wb') as file:
#     pickle.dump(jobs, file)

In [79]:
for i, job in enumerate(jobs):
    try:
        print('Data', i, job['Type'], 'OK!')
    except KeyError:
        job['Type'] = None
        print(i, job['Title'], '> "add None job type."')

Data 0 Di Kantor OK!
Data 1 Di Kantor OK!
Data 2 Di Kantor OK!
Data 3 Gabungan OK!
Data 4 Di Kantor OK!
Data 5 Di Kantor OK!
Data 6 Gabungan OK!
Data 7 Di Kantor OK!
Data 8 Di Kantor OK!
Data 9 Di Kantor OK!
Data 10 Di Kantor OK!
Data 11 Di Kantor OK!
Data 12 Di Kantor OK!
Data 13 Di Kantor OK!
Data 14 Di Kantor OK!
Data 15 Gabungan OK!
Data 16 Di Kantor OK!
Data 17 Di Kantor OK!
Data 18 Di Kantor OK!
Data 19 Gabungan OK!
Data 20 None OK!
Data 21 Di Kantor OK!
Data 22 Di Kantor OK!
Data 23 Jarak Jauh OK!
Data 24 Gabungan OK!
Data 25 Di Kantor OK!
Data 26 Di Kantor OK!
Data 27 Di Kantor OK!
Data 28 Gabungan OK!
Data 29 Di Kantor OK!
Data 30 Gabungan OK!
Data 31 Di Kantor OK!
Data 32 Gabungan OK!
Data 33 None OK!
Data 34 None OK!
Data 35 Gabungan OK!
Data 36 Di Kantor OK!
Data 37 None OK!
Data 38 Gabungan OK!
Data 39 Di Kantor OK!
Data 40 Di Kantor OK!
Data 41 None OK!
Data 42 Di Kantor OK!
Data 43 Gabungan OK!
Data 44 Gabungan OK!
Data 45 Gabungan OK!
Data 46 Gabungan OK!
Data 47 Jarak 

In [26]:
df_job = pd.DataFrame({
    'Title': [job['Title'] for job in jobs],
    'Company': [job['Company'] for job in jobs],
    'Area': [job['Area'] for job in jobs],
    'Type': [job['Type'] for job in jobs],
    'Skills': [job['Skills'] for job in jobs],
    'Description': [job['Description'] for job in jobs],
})

In [29]:
df_job.isna().sum()

Title          0
Company        0
Area           0
Type           0
Skills         0
Description    0
dtype: int64

In [28]:
df_job.fillna('', inplace=True)

In [30]:
df_job.Description = df_job.Description.map(clean_text)

c:\Users\eats\projects\eris_job\FlaskApp\transform.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, features='lxml').get_text(separator=' ')


In [31]:
df_job.Skills = df_job.Skills.apply(lambda x: ' '.join(x) if x != '' else '')

In [32]:
df_job.Skills = df_job.Skills.map(str.lower).map(maintain_alphanum)

In [33]:
df_job.Title = df_job.Title.map(str.lower).map(maintain_alphanum)

In [34]:
df_job.head(1)

,Title,Company,Area,Type,Skills,Description
0,senior 3d civil designer,Worley,"Sulawesi Barat, Indonesia",Di Kantor,3d computer aided design 3d cad autodesk civil 3d civil engineering design civil engineering drafting computer aided design cad constructability earthworks engineering drawings health and safety executive hse representational state transfer rest,tentang pekerjaan ini more than 50 000 people 49 countries over 120 office locations we re not just engineers we re global team of data scientists consultants construction workers and innovators all working to create better tomorrow every day we help customers in energy chemicals and resources get one step closer to solving our planet toughest issues climate change the energy transition digital transformation and how we can deliver more sustainable world whatever your ambition there path for you here and there no barrier to your potential career success for more information visit worley com key accountabilities responsible for safe working practices both personally and in their designs using the worley framework ensure they understand and apply the worley quality management system ensure design work conforms to applicable specifications codes and standards ensure checking procedures are adhered to before issue of drawings and documents to ensure no rework due to errors in design prepare engineering drawings and documentation promote interdisciplinary communications on projects ensure project scope budgets and schedules for design and drafting work are effectively communicated and met on projects control the allocation of drafting work to junior designers in the project mentoring and supervision of junior designers requirements degree diploma of engineering in the relevant field or equivalent minimum 5 years experience in 3d civil design for earthworks knowledge in design for constructability commissioning operations and maintenance have the experience and skills to set the standard and drive the rest of the team to excel in hse competent in civil 3d cad software excellent communication skills work as part of multi discipline team excellent time management skills and the ability to work autonomously must be able to relocate to project site to work what we offer you providing an agile culture and challenging work opportunities results in positive work environment our people enjoy competitive compensation packages and fun personal collaborative and safe working environment we value equal employment opportunity and are committed to promoting fairness equality and diversity our policy is to conduct background checks for all candidates who accept an offer of employment with us if you have the passion and talent to keep up it time to test the limits of what you can become find your future with worley company worley primary location idn sulawesi job civil schedule full time employment type employee job level executive job posting apr 17 2023 unposting date may 17 2023 reporting manager title study manager


In [35]:
df_job['Text'] = df_job.Skills + ' ' + df_job.Description

In [36]:
id_stopwords = stopwords.words('indonesian')
en_stopwords = stopwords.words('english')

In [37]:
nltk_stopwords = id_stopwords + en_stopwords

In [38]:
df_job.Text = df_job.Text.apply(lambda x: ' '.join([i for i in x.split() if i not in nltk_stopwords]))

In [42]:
df_job.head(1)

,Title,Company,Area,Type,Skills,Description,Text
0,senior 3d civil designer,Worley,"Sulawesi Barat, Indonesia",Di Kantor,3d computer aided design 3d cad autodesk civil 3d civil engineering design civil engineering drafting computer aided design cad constructability earthworks engineering drawings health and safety executive hse representational state transfer rest,tentang pekerjaan ini more than 50 000 people 49 countries over 120 office locations we re not just engineers we re global team of data scientists consultants construction workers and innovators all working to create better tomorrow every day we help customers in energy chemicals and resources get one step closer to solving our planet toughest issues climate change the energy transition digital transformation and how we can deliver more sustainable world whatever your ambition there path for you here and there no barrier to your potential career success for more information visit worley com key accountabilities responsible for safe working practices both personally and in their designs using the worley framework ensure they understand and apply the worley quality management system ensure design work conforms to applicable specifications codes and standards ensure checking procedures are adhered to before issue of drawings and documents to ensure no rework due to errors in design prepare engineering drawings and documentation promote interdisciplinary communications on projects ensure project scope budgets and schedules for design and drafting work are effectively communicated and met on projects control the allocation of drafting work to junior designers in the project mentoring and supervision of junior designers requirements degree diploma of engineering in the relevant field or equivalent minimum 5 years experience in 3d civil design for earthworks knowledge in design for constructability commissioning operations and maintenance have the experience and skills to set the standard and drive the rest of the team to excel in hse competent in civil 3d cad software excellent communication skills work as part of multi discipline team excellent time management skills and the ability to work autonomously must be able to relocate to project site to work what we offer you providing an agile culture and challenging work opportunities results in positive work environment our people enjoy competitive compensation packages and fun personal collaborative and safe working environment we value equal employment opportunity and are committed to promoting fairness equality and diversity our policy is to conduct background checks for all candidates who accept an offer of employment with us if you have the passion and talent to keep up it time to test the limits of what you can become find your future with worley company worley primary location idn sulawesi job civil schedule full time employment type employee job level executive job posting apr 17 2023 unposting date may 17 2023 reporting manager title study manager,3d computer aided design 3d cad autodesk civil 3d civil engineering design civil engineering drafting computer aided design cad constructability earthworks engineering drawings health safety executive hse representational state transfer rest pekerjaan 50 000 people 49 countries 120 office locations engineers global team data scientists consultants construction workers innovators working create better tomorrow every day help customers energy chemicals resources get one step closer solving planet toughest issues climate change energy transition digital transformation deliver sustainable world whatever ambition path barrier potential career success information visit worley com key accountabilities responsible safe working practices personally designs using worley framework ensure understand apply worley quality management system ensure design work conforms applicable specifications codes standards ensure checking procedures adhered issue drawings documents ensure rework due er

In [51]:
dataset = df_job.Text.map(str.split).values

data = []
for word in dataset:
    data.append(word)

def tagged_document(list_of_listwords, tags=None):
    if tags is not None:
        for x, listwords in zip(tags, list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])
    else:
        for x, listwords in enumerate(list_of_listwords):
            yield models.doc2vec.TaggedDocument(listwords, [x])

data_train = list(tagged_document(data, df_job.Title.values))


In [52]:
d2v_model = models.doc2vec.Doc2Vec(vector_size=120, min_count=10, epochs=60, window=3)
d2v_model.build_vocab(data_train)

2023-05-03 09:14:36,806 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d120,n5,w3,mc10,s0.001,t3>', 'datetime': '2023-05-03T09:14:36.806789', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2023-05-03 09:14:36,806 : INFO : collecting all words and their counts
2023-05-03 09:14:36,807 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2023-05-03 09:14:36,932 : INFO : collected 18707 word types and 2765 unique tags from a corpus of 5125 examples and 1002456 words
2023-05-03 09:14:36,933 : INFO : Creating a fresh vocabulary
2023-05-03 09:14:36,957 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=10 retains 7447 unique words (39.81% of original 18707, drops 11260)', 'datetime': '2023-05-03T09:14:36.957890', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021,

In [58]:
list(d2v_model.wv.index_to_key)

['experience',
 'business',
 'team',
 'sales',
 'work',
 'pekerjaan',
 'management',
 'skills',
 'development',
 'company',
 'data',
 'job',
 'product',
 'customer',
 'marketing',
 'indonesia',
 'knowledge',
 'strong',
 'support',
 'working',
 'years',
 'services',
 'requirements',
 'end',
 'new',
 'ensure',
 'design',
 'ability',
 'service',
 'communication',
 'project',
 'solutions',
 'customers',
 'good',
 'technical',
 'market',
 'develop',
 'related',
 'technology',
 'english',
 'degree',
 'software',
 'provide',
 'products',
 'teams',
 'key',
 'time',
 'industry',
 'including',
 'build',
 'process',
 'responsibilities',
 'description',
 'role',
 'responsible',
 'engineering',
 'understanding',
 'performance',
 'full',
 'maintain',
 'quality',
 'web',
 'environment',
 'us',
 'information',
 'people',
 'based',
 'well',
 'manage',
 'growth',
 'best',
 'across',
 'application',
 'world',
 'high',
 'opportunity',
 'bachelor',
 'operations',
 'excellent',
 'client',
 'global',
 'komun

In [59]:
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

2023-05-03 09:17:14,076 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 3 workers on 7447 vocabulary and 120 features, using sg=0 hs=0 sample=0.001 negative=5 window=3 shrink_windows=True', 'datetime': '2023-05-03T09:17:14.076596', 'gensim': '4.3.1', 'python': '3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'train'}
2023-05-03 09:17:14,544 : INFO : EPOCH 0: training on 1002456 raw words (935067 effective words) took 0.5s, 2004296 effective words/s
2023-05-03 09:17:15,013 : INFO : EPOCH 1: training on 1002456 raw words (935055 effective words) took 0.5s, 2008356 effective words/s
2023-05-03 09:17:15,461 : INFO : EPOCH 2: training on 1002456 raw words (934989 effective words) took 0.4s, 2095742 effective words/s
2023-05-03 09:17:15,914 : INFO : EPOCH 3: training on 1002456 raw words (935217 effective words) took 0.5s, 2071011 effective words/s
2023-05-03 09:17:16,350 : INFO 

In [60]:
linkedin_words = d2v_model.wv.index_to_key
linkedin_vectors = d2v_model.wv.vectors

with open('data/linkedin_vectors.tsv', 'w', encoding='utf-8') as f:
    for i, word in enumerate(linkedin_words):
        vector_str = '\t'.join([str(num) for num in linkedin_vectors[i]])
        f.write(f'{vector_str}\n')

with open('data/linkedin_metadata.tsv', 'w', encoding='utf-8') as f:
    for word in linkedin_words:
        f.write(f'{word}\n')